# Pytorch lightning으로 저장한 model 제대로 불러와지는지 확인

In [15]:
import torch
from vision_transformer3 import vision_transformer3 

default_kwargs = {
                            # image
                        'image_size': 224,
                        'patch_size': 32, # if '32' in hparams.arch else 16,
                        'mode': 'base',
                        'embed_dim': 512
                    }
print(default_kwargs)
model=vision_transformer3(**default_kwargs)


pretrained_weights = '/shared/js.yun/DINO_sagemaker/ckpts/exp/epoch=48.ckpt'
state_dict = torch.load(pretrained_weights, map_location="cpu")
# checkpoint_key = 'teacher'
checkpoint_key = 'state_dict'

# for i in state_dict:
#     print(i)

if checkpoint_key is not None and checkpoint_key in state_dict:
    print(f"Take key {checkpoint_key} in provided checkpoint dict")
    state_dict = state_dict[checkpoint_key]
    # print(state_dict)

teacher_dict = {}
for k,v in state_dict.items():
    if 'teacher.backbone' in k:
        key = k.replace("teacher.backbone.", "")
        teacher_dict[key] = v

# remove `module.` prefix
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
# remove `backbone.` prefix induced by multicrop wrapper
state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
msg = model.load_state_dict(teacher_dict, strict=False)
print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))

{'image_size': 224, 'patch_size': 32, 'mode': 'base', 'embed_dim': 512}


Take key state_dict in provided checkpoint dict
Pretrained weights found at /shared/js.yun/DINO_sagemaker/ckpts/exp/epoch=48.ckpt and loaded with msg: <All keys matched successfully>


# Dataloader 좌표 합치는 부분 속도 확인

In [107]:
import numpy as np
import time

data = np.random.randint(0,2000000000, size=(50000000,2), dtype=np.int64)
# print(data)

# list append
start = time.time()
bucket = []
for i in data:
    bucket.append(i)
print(time.time()-start)
bucket = np.vstack(bucket)
print(time.time()-start)
# print(bucket[0])

# # list extend
# start = time.time()
# bucket = []
# for i in data:
#     bucket.extend([i])
# print(time.time()-start)
# bucket = np.vstack(bucket)
# # print(bucket)
# print(time.time()-start)


# # list extend
# start = time.time()
# bucket = np.empty_like(data)
# for i,j in enumerate(data):
#     bucket[i] = j
# # print(time.time()-start)
# # bucket = np.vstack(bucket)
# # print(bucket)
# print(time.time()-start)


# # list append, extend
# start = time.time()
# bucket = []
# bucket2 = []
# for i in data:
#     bucket.append(i)
#     bucket2.extend([i])
# bucket = np.vstack(bucket)
# bucket2 = np.vstack(bucket)
# # print(bucket)
# print(time.time()-start)



# numpy append
# start = time.time()
# bucket = np.array([], dtype=np.int64).reshape(0,2)
# for i in data:
#     bucket = np.vstack([bucket, i])
# # print(bucket)
# print(time.time()-start)


15.452617883682251
88.03526329994202
